In [1]:
!pip install skorch

     |████████████████████████████████| 133kB 18.1MB/s 


In [2]:
import torch.nn as nn
from skorch import NeuralNetClassifier
import torch
import torch.optim as optim
from transformers import *
import pickle
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [4]:
with open("x_train.pkl", "rb") as file:
    x_train = pickle.load(file)

with open("x_valid.pkl", "rb") as file:
    x_valid = pickle.load(file)

with open("x_test.pkl", "rb") as file:
    x_test = pickle.load(file)

with open("y_train.pkl", "rb") as file:
    y_train = pickle.load(file)

with open("y_valid.pkl", "rb") as file:
    y_valid = pickle.load(file)

with open("y_test.pkl", "rb") as file:
    y_test = pickle.load(file)

In [19]:
class FullyConnectedNet(nn.Module):
    def __init__(self):
        super(FullyConnectedNet, self).__init__()

        self.fully_connected_layer = nn.Sequential(
            nn.Linear(514, 380),            
            nn.Tanh(),
            nn.Dropout(p=0.8),
            nn.Linear(380, 240),            
            nn.Tanh(),
            nn.Dropout(p=0.5),
            nn.Linear(240, 120),            
            nn.Tanh(),
            nn.Dropout(p=0.4),
            nn.Linear(120, 64),            
            nn.Tanh(),
            nn.Dropout(p=0.3),
            nn.Linear(64, 16),            
            nn.Tanh(),
            nn.Dropout(p=0.1),
            nn.Linear(16, 2),
        )

    def forward(self, x):
        x = x.view(x.shape[0], 1, -1)

        x = x.view(x.size(0), -1)
        x = self.fully_connected_layer(x)

        return x

In [20]:
net = NeuralNetClassifier(
        FullyConnectedNet,
        max_epochs=30,
        lr=0.0001,
        batch_size=1024,
        verbose=4,
        criterion=nn.CrossEntropyLoss,
        optimizer=optim.Adam,
        device=device)

In [21]:
pipe = Pipeline(steps=[
    ('RoleLineMerge', RoleLineMergeTransformer()),
    ('DropUselessColumns', DropUselessColumnsTransformer(extra_input=[ExtraInputType.FIRST_BLOOD, ExtraInputType.FIRST_DRAGON, ExtraInputType.FIRST_HERALD, ExtraInputType.FIRST_TOWER])),
    ('MakeVector', MakeVectorTransformer(top_n=50, augmented=False)),
    ('Model', net)
])

init called
init called
init called


In [22]:
pipe.fit(x_train, y_train['Win'].astype('int64'))

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6970       0.4944        0.6916  0.2848
      2        0.6878       0.6586        0.6821  0.2846
      3        0.6785       0.6089        0.6719  0.2862
      4        0.6673       0.6513        0.6567  0.2911
      5        0.6493       0.6674        0.6348  0.3405
      6        0.6296       0.6697        0.6178  0.2887
      7        0.6176       0.6702        0.6129  0.2956
      8        0.6163       0.6710        0.6115  0.2959
      9        0.6121       0.6731        0.6105  0.3286
     10        0.6072       0.6722        0.6099  0.3668
     11        0.6066       0.6708        0.6095  0.4184
     12        0.6080       0.6724        0.6092  0.3691
     13        0.6054       0.6727        0.6090  0.3557
     14        0.6048       0.6747        0.6091  0.2866
     15        0.6055       0.6748        0.6091  0.2866
     16        0.6038       0.6

Pipeline(memory=None,
         steps=[('RoleLineMerge', RoleLineMergeTransformer()),
                ('DropUselessColumns',
                 DropUselessColumnsTransformer(extra_input=[<ExtraInputType.FIRST_BLOOD: 1>,
                                                            <ExtraInputType.FIRST_DRAGON: 2>,
                                                            <ExtraInputType.FIRST_HERALD: 4>,
                                                            <ExtraInputType.FIRST_TOWER: 3>])),
                ('MakeVector',
                 MakeVectorTransformer(augmented=False, top_n=50)),
                ('Model',
                 <class 'skor...
      (6): Linear(in_features=240, out_features=120, bias=True)
      (7): Tanh()
      (8): Dropout(p=0.4, inplace=False)
      (9): Linear(in_features=120, out_features=64, bias=True)
      (10): Tanh()
      (11): Dropout(p=0.3, inplace=False)
      (12): Linear(in_features=64, out_features=16, bias=True)
      (13): Tanh()
      (14): 

In [23]:
predictions = pipe.predict(x_test)
print(accuracy_score(y_true=y_test['Win'].astype(float), y_pred=predictions))

0.6804314329738058
